# 自動到 "教師研習網查詢有興趣的研習"

由於教師研習網並沒有提供過濾有興趣的研習的通知，每次都通知一大堆不會參加的研習，到最後就不想看了。

我們利用瀏覽器爬蟲來完成這個任務吧。讓自己更輕鬆地獲知研習訊息。


# 首先安裝必要的套件


## 安裝 selenium

    selenium 是一個自動測試工具

In [11]:
!pip install selenium

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# 下載瀏覽器控制器

 [chromedriver.exe](files/chromedriver.exe)
 
 將 chromedriver.exe 複製到 C:\\Users\\jiangsir\\Anaconda3\\  以便未來使用

In [1]:
## 找到使用的是哪一個 python

import sys
sys.executable

'C:\\Users\\jiangsir\\Anaconda3\\python.exe'

# 見證神奇的時刻

### 你會見到瀏覽器自行打開，連到指定的網站，並且這個瀏覽器會顯示被自動測試程式控制。

![](images/Selenium.PNG)

In [3]:
from selenium import webdriver
# 打開瀏覽器
driver = webdriver.Chrome()

# 連到我想去的網站
driver.get("http://zerojudge.tw")

#關閉瀏覽器
#driver.close()

# 連線到教師研習網進行單一條件查詢

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
import requests
import pandas as pd

# executable_path="C:\\Users\\jiangsir\\Anaconda3\\chromedriver.exe"

driver = webdriver.Chrome()
driver.implicitly_wait(30)
verificationErrors = []
accept_next_alert = True

driver.get("https://www4.inservice.edu.tw/NAPP/OpenQuery.aspx")
driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").click()
driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").clear()
driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").send_keys("人工智慧")
driver.find_element_by_xpath("(.//*[normalize-space(text()) and normalize-space(.)='顯示更多方式'])[1]/following::span[1]").click()
element = driver.find_element_by_xpath('//*[@id="ctl00_CPH_Content_RG_SearchResult_ctl00"]')
html = element.get_attribute('outerHTML')
df = pd.read_html(html)
driver.close()
df[0]

,課程代碼,研習名稱,開始日期,結束日期,辦理研習單位,列表
0,2639128,108商管群科中心學校教師研習-以Python數據擷取與分析暨AI人工智慧應用為例,2019/07/10,2019/07/10,臺中市 市立臺中家商,26391282019/07/10~ 2019/07/10 108商管群科中心學校教師研習...
1,2651912,Ai人工智慧視覺機器人平臺研習,2019/07/22,2019/07/26,臺南市 國立臺南高工,26519122019/07/22~ 2019/07/26 Ai人工智慧視覺機器人平臺研習...
2,2667788,人工智慧與創意物聯的領航,2019/07/22,2019/08/02,南投縣 國立暨大附中,26677882019/07/22~ 2019/08/02 人工智慧與創意物聯的領航 南...
3,2659113,人工智慧系列教師培訓工作坊,2019/07/24,2019/08/20,高雄市 市立高雄女中,26591132019/07/24~ 2019/08/20 人工智慧系列教師培訓工作坊 高...
4,2659157,(成功自造中心)人工智慧專題<影像辨識猜拳機>,2019/08/02,2019/08/02,宜蘭縣 宜蘭縣政府教育處,26591572019/08/02~ 2019/08/02 (成功自造中心)人工智慧專題<...
5,2662002,人工智慧發展趨勢與AI影像辨識範例班,2019/08/10,2019/08/10,彰化縣 國立彰化師範大學,26620022019/08/10~ 2019/08/10 人工智慧發展趨勢與AI影像辨識...
6,1 每頁筆數： select 6 items in 1 pages,NaN,NaN,NaN,NaN,NaN


# 連線到教師研習網進行多條件查詢

In [5]:

import time, os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
import requests
import pandas as pd
import smtplib
from email.mime.text import MIMEText

def query(driver, keyword):
    driver.get("https://www4.inservice.edu.tw/NAPP/OpenQuery.aspx")
    driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").click()
    driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").clear()
    driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").send_keys(keyword)
    driver.find_element_by_xpath("(.//*[normalize-space(text()) and normalize-space(.)='顯示更多方式'])[1]/following::span[1]").click()
    element = driver.find_element_by_xpath('//*[@id="ctl00_CPH_Content_RG_SearchResult_ctl00"]')
    html = element.get_attribute('outerHTML')
    return pd.read_html(html)

# executable_path="C:\\Users\\jiangsir\\Anaconda3\\chromedriver.exe"
driver = webdriver.Chrome()

driver.implicitly_wait(30)
verificationErrors = []
accept_next_alert = True

keywords = ["人工智慧", "物聯網", "大數據"]

df = pd.DataFrame()
for keyword in keywords:
    df1 = query(driver, keyword)
    if df.empty:
        df = df1[0]
    else:
        df = pd.concat([df, df1[0]])

#driver.close()
driver.quit()

df


,課程代碼,研習名稱,開始日期,結束日期,辦理研習單位,列表
0,2639128,108商管群科中心學校教師研習-以Python數據擷取與分析暨AI人工智慧應用為例,2019/07/10,2019/07/10,臺中市 市立臺中家商,26391282019/07/10~ 2019/07/10 108商管群科中心學校教師研習...
1,2651912,Ai人工智慧視覺機器人平臺研習,2019/07/22,2019/07/26,臺南市 國立臺南高工,26519122019/07/22~ 2019/07/26 Ai人工智慧視覺機器人平臺研習...
2,2667788,人工智慧與創意物聯的領航,2019/07/22,2019/08/02,南投縣 國立暨大附中,26677882019/07/22~ 2019/08/02 人工智慧與創意物聯的領航 南...
3,2659113,人工智慧系列教師培訓工作坊,2019/07/24,2019/08/20,高雄市 市立高雄女中,26591132019/07/24~ 2019/08/20 人工智慧系列教師培訓工作坊 高...
4,2659157,(成功自造中心)人工智慧專題<影像辨識猜拳機>,2019/08/02,2019/08/02,宜蘭縣 宜蘭縣政府教育處,26591572019/08/02~ 2019/08/02 (成功自造中心)人工智慧專題<...
5,2662002,人工智慧發展趨勢與AI影像辨識範例班,2019/08/10,2019/08/10,彰化縣 國立彰化師範大學,26620022019/08/10~ 2019/08/10 人工智慧發展趨勢與AI影像辨識...
6,1 每頁筆數： select 6 items in 1 pages,NaN,NaN,NaN,NaN,NaN
0,2659149,(成功自造中心)物聯網木質時鐘,2019/07/11,2019/07/11,宜蘭縣 宜蘭縣政府教育處,26591492019/07/11~ 2019/07/11 (成功自造中心)物聯網木質時鐘...
1,2659888,(成功自造中心)聯發科物聯網linkit-7697套件基礎研習,2019/07/12,2019/07/12,宜蘭縣 宜蘭縣政府教育處,26598882019/07/12~ 2019/07/12 (成功自造中心)聯發科物聯網l...
2,2660306,【區域推廣中心】AIoT智能物聯網控制板,2019/07/25,2019/07/25,臺北市 臺北市立永春高級中學,26603062019/07/25~ 2019/07/25 【區域推廣中心】AIoT智能物...


# 將查到的資訊合併，取聯集

In [56]:

import time, os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
import requests
import pandas as pd
import smtplib
from email.mime.text import MIMEText

def query(driver, keyword):
    driver.get("https://www4.inservice.edu.tw/NAPP/OpenQuery.aspx")
    driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").click()
    driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").clear()
    driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").send_keys(keyword)
    driver.find_element_by_xpath("(.//*[normalize-space(text()) and normalize-space(.)='顯示更多方式'])[1]/following::span[1]").click()
    element = driver.find_element_by_xpath('//*[@id="ctl00_CPH_Content_RG_SearchResult_ctl00"]')
    html = element.get_attribute('outerHTML')
    df_html = pd.read_html(html)[0]
    df_html.drop(df_html.tail(1).index, inplace=True) 
    return df_html

# executable_path="C:\\Users\\jiangsir\\Anaconda3\\chromedriver.exe"
driver = webdriver.Chrome()

driver.implicitly_wait(30)
verificationErrors = []
accept_next_alert = True

keywords = ["人工智慧", "物聯網", "大數據"]

df = pd.DataFrame()

for keyword in keywords:
    df1 = query(driver, keyword)
    if df.empty:
        df = df1
    else:
        df = pd.concat([df, df1])

#driver.close()
driver.quit()


df = df.drop_duplicates(['課程代碼'])
df = df.sort_values(['開始日期'],ascending=[1])
df


,課程代碼,研習名稱,開始日期,結束日期,辦理研習單位,列表
0,2587435,縣級「新興科技」教師增能工作坊-AI人工智慧大數據(場次二),2019/04/27,2019/06/15,彰化縣 縣立花壇國中,25874352019/04/27~ 2019/06/15 縣級「新興科技」教師增能工作坊...
0,2608404,NKNUScratch3.0＋電動車+物聯網衛星基地認證班,2019/04/27,2019/04/27,高雄市 國立高雄師範大學,26084042019/04/27~ 2019/04/27 NKNUScratch3.0＋...
1,2612899,第四梯次：智慧物聯網的認識與應用研習,2019/04/29,2019/04/29,南投縣 縣立康壽國小,26128992019/04/29~ 2019/04/29 第四梯次：智慧物聯網的認識與應...
1,2615610,AI人工智慧語音控制與智慧小車(生活科技),2019/04/30,2019/04/30,臺中市 私立明道高中,26156102019/04/30~ 2019/04/30 AI人工智慧語音控制與智慧小車...
2,2594808,高雄市自造教育及科技中心 自造教育課程研發與推廣系列課程教師培訓 WebBit與物聯網教師研...,2019/05/02,2019/05/02,高雄市 市立中山國中,25948082019/05/02~ 2019/05/02 高雄市自造教育及科技中心 自造...
3,2598363,程式控制發展社群分享講座(物聯網運用融入教學),2019/05/03,2019/05/03,基隆市 私立二信高中,25983632019/05/03~ 2019/05/03 程式控制發展社群分享講座(物聯...
1,2583070,108年春季分享大師視野講座：從醫療大數據談臺灣公共衛生,2019/05/03,2019/05/03,臺中市 國立自然科學博物館,25830702019/05/03~ 2019/05/03 108年春季分享大師視野講座：...
4,2598415,Webduino 與物聯網教學活動-基礎研習,2019/05/07,2019/05/07,臺中市 市立僑仁國小,25984152019/05/07~ 2019/05/07 Webduino 與物聯網教學...
2,2621439,「人工智慧咖啡GO：水、火、風、土、氣與咖啡晶球」,2019/05/08,2019/05/08,臺南市 國立臺南女中,26214392019/05/08~ 2019/05/08 「人工智慧咖啡GO：水、火、風...
5,2606874,「Python程式在物聯網的應用 」教師專業成長研習,2019/05/08,2019/05/08,臺南市 國立臺南高商,26068742019/05/08~ 2019/05/08 「Python程式在物聯網的應...


# 判斷哪些是新增的研習

### 希望能夠清楚列出本次搜尋所發現的新增研習並且通知，會更友善。
### 那就必須把上一次的搜尋結果記錄下來以便下一次比對。由於資料量小，不需要動用到資料庫，直接以 CSV 儲存 即可。

In [59]:

import time, os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
import requests
import pandas as pd
import smtplib
from email.mime.text import MIMEText

def query(driver, keyword):
    driver.get("https://www4.inservice.edu.tw/NAPP/OpenQuery.aspx")
    driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").click()
    driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").clear()
    driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").send_keys(keyword)
    driver.find_element_by_xpath("(.//*[normalize-space(text()) and normalize-space(.)='顯示更多方式'])[1]/following::span[1]").click()
    element = driver.find_element_by_xpath('//*[@id="ctl00_CPH_Content_RG_SearchResult_ctl00"]')
    html = element.get_attribute('outerHTML')
    df_html = pd.read_html(html)[0]
    df_html.drop(df_html.tail(1).index, inplace=True) 
    return df_html

# executable_path="C:\\Users\\jiangsir\\Anaconda3\\chromedriver.exe"
driver = webdriver.Chrome()

driver.implicitly_wait(30)
verificationErrors = []
accept_next_alert = True

keywords = ["人工智慧", "物聯網", "大數據"]

df = pd.DataFrame()

for keyword in keywords:
    df1 = query(driver, keyword)
    if df.empty:
        df = df1
    else:
        df = pd.concat([df, df1])

#driver.close()
driver.quit()


df = df.drop_duplicates(['課程代碼'])
df = df.sort_values(['開始日期'],ascending=[1])

df.to_csv('TeacherCourses.csv')
df

,課程代碼,研習名稱,開始日期,結束日期,辦理研習單位,列表
0,2587435,縣級「新興科技」教師增能工作坊-AI人工智慧大數據(場次二),2019/04/27,2019/06/15,彰化縣 縣立花壇國中,25874352019/04/27~ 2019/06/15 縣級「新興科技」教師增能工作坊...
0,2608404,NKNUScratch3.0＋電動車+物聯網衛星基地認證班,2019/04/27,2019/04/27,高雄市 國立高雄師範大學,26084042019/04/27~ 2019/04/27 NKNUScratch3.0＋...
1,2612899,第四梯次：智慧物聯網的認識與應用研習,2019/04/29,2019/04/29,南投縣 縣立康壽國小,26128992019/04/29~ 2019/04/29 第四梯次：智慧物聯網的認識與應...
1,2615610,AI人工智慧語音控制與智慧小車(生活科技),2019/04/30,2019/04/30,臺中市 私立明道高中,26156102019/04/30~ 2019/04/30 AI人工智慧語音控制與智慧小車...
2,2594808,高雄市自造教育及科技中心 自造教育課程研發與推廣系列課程教師培訓 WebBit與物聯網教師研...,2019/05/02,2019/05/02,高雄市 市立中山國中,25948082019/05/02~ 2019/05/02 高雄市自造教育及科技中心 自造...
3,2598363,程式控制發展社群分享講座(物聯網運用融入教學),2019/05/03,2019/05/03,基隆市 私立二信高中,25983632019/05/03~ 2019/05/03 程式控制發展社群分享講座(物聯...
1,2583070,108年春季分享大師視野講座：從醫療大數據談臺灣公共衛生,2019/05/03,2019/05/03,臺中市 國立自然科學博物館,25830702019/05/03~ 2019/05/03 108年春季分享大師視野講座：...
4,2598415,Webduino 與物聯網教學活動-基礎研習,2019/05/07,2019/05/07,臺中市 市立僑仁國小,25984152019/05/07~ 2019/05/07 Webduino 與物聯網教學...
2,2621439,「人工智慧咖啡GO：水、火、風、土、氣與咖啡晶球」,2019/05/08,2019/05/08,臺南市 國立臺南女中,26214392019/05/08~ 2019/05/08 「人工智慧咖啡GO：水、火、風...
5,2606874,「Python程式在物聯網的應用 」教師專業成長研習,2019/05/08,2019/05/08,臺南市 國立臺南高商,26068742019/05/08~ 2019/05/08 「Python程式在物聯網的應...


# 把査到的資訊寄信給自己

以下為 mailer.py 程式，可放入排程，讓 windows 定時執行。

In [57]:
import time, os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
import requests
import pandas as pd
import smtplib
from email.mime.text import MIMEText




def mailer(subject, html, mailto='jiangzero@gmail.com', mailfrom='jiangsir@tea.nknush.kh.edu.tw', passwd='igajmuqzhnhuyzil'):
    mime=MIMEText(html, "html", "utf-8")
    mime["Subject"]=subject
    mime["From"]="教師研習自動通知"
    mime["To"]=""
    msg=mime.as_string()

    smtpssl=smtplib.SMTP_SSL("smtp.gmail.com", 465) 
    smtpssl.ehlo()
    smtpssl.login(mailfrom, passwd)

    to_addr=[mailto]
    status=smtpssl.sendmail(mailfrom, to_addr, msg)
    if status=={}:
        print("郵件傳送成功!")
    else:
        print("郵件傳送失敗!")
    smtpssl.quit()    

def query(driver, keyword):
    driver.get("https://www4.inservice.edu.tw/NAPP/OpenQuery.aspx")
    driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").click()
    driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").clear()
    driver.find_element_by_id("ctl00_CPH_Content_RTB_CourseName").send_keys(keyword)
    driver.find_element_by_xpath("(.//*[normalize-space(text()) and normalize-space(.)='顯示更多方式'])[1]/following::span[1]").click()
    element = driver.find_element_by_xpath('//*[@id="ctl00_CPH_Content_RG_SearchResult_ctl00"]')
    html = element.get_attribute('outerHTML')
    df_html = pd.read_html(html)[0]
    df_html.drop(df_html.tail(1).index, inplace=True) 
    return df_html

driver = webdriver.Chrome()
driver.implicitly_wait(30)
verificationErrors = []
accept_next_alert = True

keywords = ["人工智慧", "物聯網", "大數據"]

df = pd.DataFrame()
for keyword in keywords:
    df1 = query(driver, keyword)
    if df.empty:
        df = df1
    else:
        df = pd.concat([df, df1])
#driver.close()
driver.quit()

df = df.drop_duplicates(['課程代碼'])
df = df.sort_values(['開始日期'],ascending=[1])

mailer("教師研習網篩選通知("+str(keywords)+")", df.to_html())



郵件傳送成功!
